## Installations

In [1]:
!pip install google-cloud-aiplatform
!pip install --user datasets
!pip install --user google-cloud-pipeline-components

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.5 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires 

## Restarting the Kernel

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

## Importing Libraries and Connecting to Google Cloud

In [1]:
from google.cloud import aiplatform
from google.colab import auth as google_auth

In [2]:
google_auth.authenticate_user()

In [3]:
import vertexai

PROJECT_ID = "able-analyst-431912-a9"
vertexai.init(project = PROJECT_ID, location = "us-central1")

In [4]:
region = "us-central1"
REGION = "us-central1"
project_id = "able-analyst-431912-a9"

In [5]:
! gcloud config set project {project_id}

Updated property [core/project].


In [6]:
# Import the necessary Libraries
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings('ignore')
import vertexai
vertexai.init(project = PROJECT_ID, location = "us-central1")
import kfp
import sys
import uuid
import json
import vertexai
import pandas as pd
from google.auth import default
from datasets import load_dataset
from google.cloud import aiplatform
from vertexai.preview.language_models import TextGenerationModel, EvaluationTextSummarizationSpec

In [7]:
BUCKET_NAME = "myfinetuningdata"
BUCKET_URI = f"gs://my-finetuning-data/train.jsonl"
REGION = "us-central1"

In [34]:
df = pd.read_json("/content/train.jsonl", lines = True)
df.head()

,input_text,output_text
0,This product is amazing! I love it.,"Amazing product, highly loved."
1,The food was terrible and service was slow.,Bad food and slow service.
2,"It was an okay experience, nothing special.",An average experience.
3,Absolutely fantastic performance by the team!,Team performed fantastically.
4,"I wouldn't recommend this to anyone, very disa...","Not recommended, very disappointing."


In [35]:
df.shape

(15, 2)

In [40]:

import time

import vertexai
from vertexai.tuning import sft

# TODO(developer): Update and un-comment below line
# PROJECT_ID = "your-project-id"
vertexai.init(project=PROJECT_ID, location="us-central1")

sft_tuning_job = sft.train(
    source_model="gemini-1.5-pro-002",
    train_dataset="gs://cloud-samples-data/ai-platform/generative_ai/gemini-1_5/text/sft_train_data.jsonl",
    epochs = 2
)

# Polling for job completion
while not sft_tuning_job.has_ended:
    time.sleep(60)
    sft_tuning_job.refresh()

print(sft_tuning_job.tuned_model_name)
print(sft_tuning_job.tuned_model_endpoint_name)
print(sft_tuning_job.experiment)
# Example response:
# projects/123456789012/locations/us-central1/models/1234567890@1
# projects/123456789012/locations/us-central1/endpoints/123456789012345
# <google.cloud.aiplatform.metadata.experiment_resources.Experiment object at 0x7b5b4ae07af0>

INFO:vertexai.tuning._tuning:Creating SupervisedTuningJob
INFO:vertexai.tuning._tuning:SupervisedTuningJob created. Resource name: projects/229487045645/locations/us-central1/tuningJobs/5307663100554510336
INFO:vertexai.tuning._tuning:To use this SupervisedTuningJob in another session:
INFO:vertexai.tuning._tuning:tuning_job = sft.SupervisedTuningJob('projects/229487045645/locations/us-central1/tuningJobs/5307663100554510336')
INFO:vertexai.tuning._tuning:View Tuning Job:
https://console.cloud.google.com/vertex-ai/generative/language/locations/us-central1/tuning/tuningJob/5307663100554510336?project=229487045645


KeyboardInterrupt: 

In [41]:
tuned_model_name = sft_tuning_job.("Summarize this text Hello world, how are you?")

AttributeError: 'SupervisedTuningJob' object has no attribute 'predict'

In [27]:
import vertexai
from vertexai.generative_models import GenerativeModel

# TODO(developer): Update and un-comment below line
# PROJECT_ID = "your-project-id"
vertexai.init(project=PROJECT_ID, location="us-central1")

model = GenerativeModel("gemini-1.5-flash-002")

response = model.generate_content(
    "What's a good name for a flower shop that specializes in selling bouquets of dried flowers?"
)

print(response.text)

**Evocative & Romantic:**

* Everlasting Blooms
* Whispers of Autumn
* Dried & Delightful
* The Petal Archive
* Timeless Blossoms
* Botanical Echoes
* Bloom & Preserve
* The Dried Flower Atelier
* Sun-Kissed Petals
* Forever in Bloom

**Modern & Chic:**

* The Bloom Room (simple and memorable)
* Botanical Hues
* The Dried Bouquet Co.
* Everlasting Flora
* Petal & Stem
* Flora & Co.
* The Preserved Petal
* Botanical Legacy


**Unique & Playful:**

* Poppy & Thistle (referencing specific dried flowers)
* Paper & Petal
* The Memory Garden
* Dried & Wired
* The Flower Alchemist


**Tips for choosing:**

* **Check for availability:** Make sure the name isn't already in use (website domain, social media handles).
* **Keep it short and memorable:**  Easy to recall and share.
* **Reflect your brand:**  Choose a name that aligns with your shop's aesthetic and target audience.
* **Say it aloud:**  Does it sound good? Is it easy to pronounce?


Ultimately, the best name will depend on your person